In [4]:
import json
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.neural_networks import EstimatorQNN
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

algorithm_globals.random_seed = 12345

# We now define a two qubit unitary as defined in [3]
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target

def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc

def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)

    return target

def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc

In [ ]:

# Feature map for 64 qubits suitable for the image size
feature_map_64 = ZFeatureMap(64)

# Three alternating conv_layer and pool_layer functions
conv_layer_1 = conv_layer(64, "conv_params_1")
pool_layer_1 = pool_layer(range(0, 32), range(32, 64), "pool_params_1")

conv_layer_2 = conv_layer(64, "conv_params_2")
pool_layer_2 = pool_layer(range(0, 32), range(32, 64), "pool_params_2")

conv_layer_3 = conv_layer(64, "conv_params_3")
pool_layer_3 = pool_layer(range(0, 32), range(32, 64), "pool_params_3")

# Combine the layers to form the complete quantum circuit
circuit_64_3layers = QuantumCircuit(64)
circuit_64_3layers.compose(feature_map_64, range(64), inplace=True)
circuit_64_3layers.compose(conv_layer_1, range(64), inplace=True)
circuit_64_3layers.compose(pool_layer_1, range(64), inplace=True)
circuit_64_3layers.compose(conv_layer_2, range(64), inplace=True)
circuit_64_3layers.compose(pool_layer_2, range(64), inplace=True)
circuit_64_3layers.compose(conv_layer_3, range(64), inplace=True)
circuit_64_3layers.compose(pool_layer_3, range(64), inplace=True)


In [5]:
# Define paths to your train data folders
train_data_dir = 'test'

# Set image dimensions and batch size
img_width, img_height = 48,48
batch_size = 32

In [6]:
# Create an ImageDataGenerator for data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Generate training data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    color_mode='rgb'
)

# Extract X_train (input images) and Y_train (labels)
X_train = []
Y_train = []

for images, labels in train_generator:
    X_train.extend(images)
    Y_train.extend(labels)
    
    if len(X_train) >= len(train_generator.filenames):
        break

X_train = np.array(X_train)
Y_train = np.array(Y_train)

# Print class labels and their corresponding indices
class_indices = train_generator.class_indices
print("Class Indices:", class_indices)

# Convert labels to one-hot encoding
num_classes = len(class_indices)
Y_train_one_hot = tf.keras.utils.to_categorical(Y_train, num_classes=num_classes)

print("X_train shape:", X_train.shape)
print("Y_train_one_hot shape:", Y_train_one_hot.shape)

'''
Output:
Found 7178 images belonging to 7 classes.
Class Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
X_train shape: (7178, 48, 48, 3)
Y_train_one_hot shape: (7178, 7, 7)
'''

Found 7178 images belonging to 7 classes.
Class Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
X_train shape: (7178, 48, 48, 3)
Y_train_one_hot shape: (7178, 7, 7)


"\nOutput:\nFound 28709 images belonging to 7 classes.\nClass Indices: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}\nX_train shape: (28709, 128, 128, 3)\nY_train_one_hot shape: (28709, 7, 7)\n"

In [7]:

from sklearn.decomposition import PCA

# Confirming the images are grayscale by checking if all channels have the same values
assert np.all(X_train[:,:,:,0] == X_train[:,:,:,1]) and np.all(X_train[:,:,:,0] == X_train[:,:,:,2])

# Dropping the redundant channels
X_train = X_train[:,:,:,0]

# Flattening the images
X_train_flat = X_train.reshape(X_train.shape[0], -1)

# Applying PCA
pca = PCA(n_components=8)
X_train_pca = pca.fit_transform(X_train_flat)

X_train_pca.shape


(7178, 8)

In [8]:

# Assuming the necessary libraries and functions have been imported previously
# Extracting the shape of X_train and Y_train_one_hot
X_train_shape = X_train.shape
Y_train_one_hot_shape = Y_train_one_hot.shape

X_train_shape, Y_train_one_hot_shape


((7178, 48, 48), (7178, 7, 7))

In [ ]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [ ]:
objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)
classifier.fit(X_train, Y_train_one_hot)

# score classifier
print(f"Accuracy from the train data : {np.round(100 * classifier.score(x, y), 2)}%")

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


QiskitMachineLearningError: 'Input data has incorrect shape, last dimension is not equal to the number of inputs: 8, but got: 3.'

In [ ]:

observable_7class = [SparsePauliOp.from_list([('Z' + 'I' * 63, 1)]) for _ in range(7)]


In [ ]:

qnn_7class = EstimatorQNN(
    circuit=circuit.decompose(),
    observables=observable_7class,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
)


In [ ]:

classifier_7class = NeuralNetworkClassifier(
    qnn_7class,
    optimizer=COBYLA(maxiter=200),  # Set max iterations here
    callback=callback_graph,
)
